In [ ]:
import os, requests, rasterio, geopandas as gpd
from rasterio.mask import mask
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
from tqdm import tqdm
from rasterio.plot import plotting_extent
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap, PowerNorm, to_rgba
import overpy
from shapely.geometry import Point, LineString, Polygon
import pandas as pd
import matplotlib.patheffects as path_effects
import warnings

Population plot

In [ ]:
def download_worldpop_zambia(year=2020, out_dir="data"):
    iso3 = "ZMB"
    fn = f"{iso3.lower()}_ppp_{year}_UNadj.tif"
    url = f"https://data.worldpop.org/GIS/Population/Global_2000_2020/{year}/{iso3}/{fn}"
    os.makedirs(out_dir, exist_ok=True)
    out_path = os.path.join(out_dir, fn)

    if not os.path.exists(out_path):
        with requests.get(url, stream=True, timeout=60) as r:
            r.raise_for_status()
            with open(out_path, "wb") as f:
                for chunk in r.iter_content(chunk_size=1<<20):
                    if chunk:
                        f.write(chunk)
    return out_path

def create_population_layer(ax=None):
    tif_path = download_worldpop_zambia(2020)
    
    with rasterio.open(tif_path) as src:
        arr = src.read(1)  
        bounds = src.bounds
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(16, 14))
    
    colors = ['#f7f7f7', '#fdd49e', '#fdbb84', '#fc8d59', '#ef6548', '#d7301f', '#b30000', '#7f0000']
    cmap = LinearSegmentedColormap.from_list('population_warm', colors, N=256)
    
    vmax = np.nanpercentile(arr, 99.5)
    vmin = 0
    
    im = ax.imshow(arr, 
                   cmap=cmap, 
                   norm=PowerNorm(gamma=0.5, vmin=vmin, vmax=vmax),
                   aspect='equal',
                   extent=[bounds.left, bounds.right, bounds.bottom, bounds.top],
                   alpha=0.8)
    
    return ax, im, bounds

# Execute plot 1
ax1, im1, bounds1 = create_population_layer()
cbar1 = plt.colorbar(im1, ax=ax1, shrink=0.6, aspect=30, pad=0.02)
cbar1.set_label('Population Density', fontsize=14, fontweight='bold')
ax1.set_title('Population Density of Zambia (2020)', fontsize=24, fontweight='bold', pad=30)
ax1.set_xlabel('Longitude (°E)', fontsize=14, fontweight='bold')
ax1.set_ylabel('Latitude (°S)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

Power Lines plot

In [ ]:
def fetch_zambia_power_data():
    api = overpy.Overpass()
    overpass_query = """
    [out:json][timeout:400];
    relation["boundary"="administrative"]["name"~"Zambia"]["admin_level"="2"] -> .admin_boundary;
    .admin_boundary map_to_area -> .searchArea;
    way["power"="line"](area.searchArea) -> .power_lines;
    way["power"="cable"](area.searchArea) -> .power_cables;
    (.power_lines; .power_cables; .admin_boundary;);
    out body; >; out skel qt;
    """
    return api.query(overpass_query)

def convert_to_geodataframes(result):
    power_lines_data = []
    for way in result.ways:
        if way.tags.get('power') in ['line', 'cable']:
            coords = [(float(node.lon), float(node.lat)) for node in way.nodes]
            if len(coords) >= 2:
                power_line_data = {
                    'id': way.id,
                    'power': way.tags.get('power'),
                    'voltage': way.tags.get('voltage', ''),
                    'geometry': LineString(coords)
                }
                power_lines_data.append(power_line_data)
    
    return gpd.GeoDataFrame(power_lines_data, crs='EPSG:4326')

def get_zambia_boundary():
    try:
        world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
        zambia = world[world.name == 'Zambia'].copy()
        return zambia if len(zambia) > 0 else None
    except:
        return None

def create_power_lines_layer(ax=None):
    result = fetch_zambia_power_data()
    power_lines_gdf = convert_to_geodataframes(result)
    zambia = get_zambia_boundary()
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(16, 14))
    
    voltage_colors = {
        '330000': {'color': '#c0392b', 'width': 4, 'alpha': 0.8},
        '220000': {'color': '#e67e22', 'width': 3.5, 'alpha': 0.8},
        '132000': {'color': '#f1c40f', 'width': 3, 'alpha': 0.8},
        '88000': {'color': '#27ae60', 'width': 2.5, 'alpha': 0.8},
        '66000': {'color': '#3498db', 'width': 2, 'alpha': 0.8},
        '33000': {'color': '#9b59b6', 'width': 1.5, 'alpha': 0.8},
        '11000': {'color': '#e91e63', 'width': 1, 'alpha': 0.8}
    }
    
    if zambia is not None and len(zambia) > 0:
        bounds = zambia.total_bounds
        zambia.boundary.plot(ax=ax, color='#34495e', linewidth=2, alpha=0.6, zorder=2)
    else:
        bounds = power_lines_gdf.total_bounds if len(power_lines_gdf) > 0 else None
    
    if len(power_lines_gdf) > 0:
        for voltage in sorted(voltage_colors.keys(), key=lambda x: int(x), reverse=True):
            mask = power_lines_gdf['voltage'] == voltage
            if mask.any():
                power_lines_gdf[mask].plot(
                    ax=ax, 
                    color=voltage_colors[voltage]['color'], 
                    linewidth=voltage_colors[voltage]['width'],
                    alpha=voltage_colors[voltage]['alpha'],
                    zorder=3
                )
        
        no_voltage_mask = power_lines_gdf['voltage'].isin(['', 'unknown']) | power_lines_gdf['voltage'].isna()
        if no_voltage_mask.any():
            power_lines_gdf[no_voltage_mask].plot(
                ax=ax, color='#7f8c8d', linewidth=1, alpha=0.6, zorder=3
            )
    
    return ax, power_lines_gdf, bounds

# Execute plot 2
ax2, power_gdf, bounds2 = create_power_lines_layer()
ax2.set_title('Zambia Power Lines Network', fontsize=24, fontweight='bold', pad=30)
ax2.set_xlabel('Longitude (°E)', fontsize=14, fontweight='bold')
ax2.set_ylabel('Latitude (°S)', fontsize=14, fontweight='bold')
ax2.set_aspect('equal')
plt.tight_layout()
plt.show()

Substations Plot

In [ ]:
def fetch_zambia_substations_data():
    api = overpy.Overpass()
    overpass_query = """
    [out:json][timeout:1400];
    relation["boundary"="administrative"]["name"~"Zambia"]["admin_level"="2"] -> .admin_boundary;
    .admin_boundary map_to_area -> .searchArea;
    (
      node["power"="substation"](area.searchArea);
      way["power"="substation"](area.searchArea);
      relation["power"="substation"](area.searchArea);
    );
    out body; >; out skel qt;
    """
    return api.query(overpass_query)

def convert_substations_to_geodataframes(result):
    substations_data = []
    
    for node in result.nodes:
        if node.tags.get('power') == 'substation':
            substation_data = {
                'id': node.id,
                'type': 'node',
                'power': 'substation',
                'voltage': node.tags.get('voltage', ''),
                'name': node.tags.get('name', ''),
                'geometry': Point(float(node.lon), float(node.lat))
            }
            substations_data.append(substation_data)
    
    for way in result.ways:
        if way.tags.get('power') == 'substation':
            coords = [(float(node.lon), float(node.lat)) for node in way.nodes]
            if len(coords) >= 3:
                substation_data = {
                    'id': way.id,
                    'type': 'way',
                    'power': 'substation',
                    'voltage': way.tags.get('voltage', ''),
                    'name': way.tags.get('name', ''),
                    'geometry': Polygon(coords)
                }
                substations_data.append(substation_data)
    
    return gpd.GeoDataFrame(substations_data, crs='EPSG:4326')

def create_substations_layer(ax=None):
    result = fetch_zambia_substations_data()
    substations_gdf = convert_substations_to_geodataframes(result)
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(16, 14))
    
    if len(substations_gdf) > 0:
        point_substations = substations_gdf[substations_gdf.geometry.geom_type == 'Point'].copy()
        polygon_substations = substations_gdf[substations_gdf.geometry.geom_type.isin(['Polygon', 'MultiPolygon'])].copy()
        
        # Create 10km buffers around substations
        # Project to a metric CRS for accurate 10km buffer
        if len(point_substations) > 0:
            point_substations_proj = point_substations.to_crs(epsg=3395)  # World Mercator
            point_buffers = point_substations_proj.geometry.buffer(10000)  # 10km = 10,000m
            point_buffers_gdf = gpd.GeoDataFrame(geometry=point_buffers, crs='EPSG:3395').to_crs(epsg=4326)
            
            # Plot point buffers
            point_buffers_gdf.plot(
                ax=ax,
                facecolor='yellow',
                edgecolor='gold',
                alpha=0.3,
                linewidth=2,
                zorder=4
            )
        
        if len(polygon_substations) > 0:
            polygon_substations_proj = polygon_substations.to_crs(epsg=3395)  # World Mercator
            polygon_buffers = polygon_substations_proj.geometry.buffer(10000)  # 10km = 10,000m
            polygon_buffers_gdf = gpd.GeoDataFrame(geometry=polygon_buffers, crs='EPSG:3395').to_crs(epsg=4326)
            
            # Plot polygon buffers
            polygon_buffers_gdf.plot(
                ax=ax,
                facecolor='yellow',
                edgecolor='gold',
                alpha=0.3,
                linewidth=2,
                zorder=4
            )
            
            # Plot the original polygon substations
            polygon_substations.plot(
                ax=ax, 
                color='#000000', 
                alpha=1,
                edgecolor='#000000',
                linewidth=7,
                zorder=6
            )
        
        # Plot the point substations (original code)
        if len(point_substations) > 0:
            marker_size = 800
            
            for idx, substation in point_substations.iterrows():
                ax.scatter(
                    substation.geometry.x, substation.geometry.y,
                    s=marker_size * 2,
                    c='#000000',
                    alpha=0.2,
                    zorder=5
                )
                
                ax.scatter(
                    substation.geometry.x, substation.geometry.y,
                    s=marker_size,
                    c='#000000',
                    alpha=0.8,
                    edgecolors='white',
                    linewidth=1,
                    zorder=7
                )
    
    bounds = substations_gdf.total_bounds if len(substations_gdf) > 0 else None
    return ax, substations_gdf, bounds

# Execute plot 3
ax3, substations_gdf, bounds3 = create_substations_layer()
ax3.set_title('Zambia Electrical Substations', fontsize=24, fontweight='bold', pad=30)
ax3.set_xlabel('Longitude (°E)', fontsize=14, fontweight='bold')
ax3.set_ylabel('Latitude (°S)', fontsize=14, fontweight='bold')
ax3.set_aspect('equal')
plt.tight_layout()
plt.show()

Combined Plot

In [ ]:
def create_layered_zambia_map():
    """Create layered map with population density, power lines, and substations"""
    
    fig, ax = plt.subplots(figsize=(16, 14))
    fig.patch.set_facecolor('white')
    
    print("Adding population density layer...")
    ax, im1, bounds1 = create_population_layer(ax)
    
    print("Adding power lines layer...")
    ax, power_gdf, bounds2 = create_power_lines_layer(ax)
    
    print("Adding substations layer...")
    ax, substations_gdf, bounds3 = create_substations_layer(ax)
    
    cbar = plt.colorbar(im1, ax=ax, shrink=0.6, aspect=30, pad=0.02)
    cbar.set_label('Population Density (people per pixel)', fontsize=12, fontweight='bold')
    cbar.ax.tick_params(labelsize=10)
    
    voltage_colors = {
        '330000': {'color': '#c0392b', 'label': '330kV'},
        '220000': {'color': '#e67e22', 'label': '220kV'},
        '132000': {'color': '#f1c40f', 'label': '132kV'},
        '88000': {'color': '#27ae60', 'label': '88kV'},
        '66000': {'color': '#3498db', 'label': '66kV'},
        '33000': {'color': '#9b59b6', 'label': '33kV'},
        '11000': {'color': '#e91e63', 'label': '11kV'}
    }
    
    from matplotlib.lines import Line2D
    from matplotlib.patches import Patch
    
    legend_elements = []
    
    for voltage, props in voltage_colors.items():
        if len(power_gdf) > 0 and (power_gdf['voltage'] == voltage).any():
            legend_elements.append(Line2D([0], [0], color=props['color'], 
                                        linewidth=3, label=f"Power Line {props['label']}"))
    
    if len(substations_gdf) > 0:
        legend_elements.append(Line2D([0], [0], marker='o', color='w', 
                                    markerfacecolor='#000000', markersize=8, 
                                    label='Substations', linestyle='None'))
    
    if legend_elements:
        legend = ax.legend(handles=legend_elements, loc='upper left', 
                          fontsize=10, frameon=True, fancybox=True, shadow=True,
                          framealpha=0.9, facecolor='white', edgecolor='black',
                          title='Infrastructure Legend', title_fontsize=11)
    
    ax.set_title('Zambia: Population Density, Power Grid & Substations\nIntegrated Infrastructure Map', 
                 fontsize=20, fontweight='bold', pad=30)
    ax.set_xlabel('Longitude (°E)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Latitude (°S)', fontsize=12, fontweight='bold')
    
    ax.set_aspect('equal')
    ax.tick_params(labelsize=10)
    
    plt.tight_layout()
    return fig, ax

fig, ax = create_layered_zambia_map()
plt.show()